<a href="https://colab.research.google.com/github/danielmohansahu/M3DETR/blob/feature%2Fsahu%2Fprofiling/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M3DETR Evaluation Notebook

This notebook demonstrates evaluation of the [M3DETR](https://arxiv.org/pdf/2104.11896.pdf) 3D Object Detection model architecture and training pipeline. Specifically it supports training and inference of [Group 13's fork](https://github.com/danielmohansahu/M3DETR). All training and evaluation uses the [KITTI 3D Detection Evaluation Dataset](https://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d).

In [ ]:
# install general dependencies / tools / etc.
!python3 -m pip install tqdm
!python3 -m pip install fiftyone
!python3 -m pip install fiftyone-db-ubuntu2204

# import support tooling
from tqdm import tqdm
import fiftyone

In [ ]:
# download KITTI 3D Object Detection dataset
dataset = fiftyone.zoo.load_zoo_dataset('kitti-multiview')

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/kitti-multiview/train' if necessary


INFO:fiftyone.utils.kitti:Downloading labels to '/root/fiftyone/kitti-multiview/tmp-download/data_object_label_2.zip'


 100% |█████|   42.7Mb/42.7Mb [874.3ms elapsed, 0s remaining, 48.9Mb/s]      


INFO:eta.core.utils: 100% |█████|   42.7Mb/42.7Mb [874.3ms elapsed, 0s remaining, 48.9Mb/s]      


Extracting '/root/fiftyone/kitti-multiview/tmp-download/data_object_label_2.zip'


...

INFO:eta.core.utils: 100% |█████|   89.4Gb/89.4Gb [12.2m elapsed, 0s remaining, 181.5Mb/s]      


Extracting '/root/fiftyone/kitti-multiview/tmp-download/data_object_image_3.zip'


INFO:fiftyone.utils.kitti:Extracting '/root/fiftyone/kitti-multiview/tmp-download/data_object_image_3.zip'


INFO:fiftyone.utils.kitti:Downloading velodyne to '/root/fiftyone/kitti-multiview/tmp-download/data_object_velodyne.zip'


  39% |█/--|   83.1Gb/214.2Gb [6.3m elapsed, 9.7m remaining, 240.4Mb/s]    

In [ ]:
# install required dependencies
!pip install -q condacolab
import condacolab
condacolab.install()

conda install pytorch torchvision pytorch-cuda=12.1 -c pytorch -c nvidia
!pip install spconv-cu120 cython numpy==1.25
!git clone https://github.com/danielmohansahu/M3DETR.git
!cd M3DETR
!python setup.py develop

# clone repo